In [1]:
from pymongo import MongoClient
import pandas as pd
import json
import geojson
import os
import requests
from dotenv import load_dotenv
import time
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

## VISUALIZING ##

**IMPORTING JSON INTO DF**

    SAN FRANCISCO

In [2]:
client = MongoClient("localhost:27017")
db = client["project_3"]

In [5]:
SFstarbucks = pd.DataFrame(list(db.SFstarbucks.find()))
SFstarbucks.sample()

,_id,name,lat,lon
3,6546afef716a854e3d9692e3,Starbucks,37.788706,-122.279538


In [6]:
SFschools = pd.DataFrame(list(db.SFschools.find()))
SFschools.sample()

,_id,name,lat,lon
43,6546aff1716a854e3d969317,San Miguel Elementary School,37.391484,-122.008433


In [7]:
SFpet_grooming = pd.DataFrame(list(db.SFpet_grooming.find()))
SFpet_grooming.sample()

,_id,name,lat,lon
48,6546aff3716a854e3d96934e,San Bruno Dog Obedience School,37.616007,-122.435966


In [8]:
SFdog_parks = pd.DataFrame(list(db.SFdog_parks.find()))
SFdog_parks.sample()

,_id,name,lat,lon
27,6546aff4716a854e3d96936b,OAK Smoking Area / Animal Relief Area,37.711441,-122.210758


In [9]:
SFbars_clubs = pd.DataFrame(list(db.SFbars_clubs.find()))
SFbars_clubs.sample()

,_id,name,lat,lon
43,6546aff5716a854e3d9693ad,True Laurel,37.759466,-122.411421


In [10]:
SFvegan_vegetarian = pd.DataFrame(list(db.SFvegan_vegetarian.find()))
SFvegan_vegetarian.sample()

,_id,name,lat,lon
16,6546aff6716a854e3d9693c4,Señor Sisig,37.805386,-122.267155


    NEW YORK